In [ ]:
!rm -rf qvina
!git clone https://github.com/QVina/qvina.git
!chmod +x qvina/bin/qvina02


Cloning into 'qvina'...
remote: Enumerating objects: 526, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 526 (delta 1), reused 15 (delta 0), pack-reused 510 (from 1)
Receiving objects: 100% (526/526), 27.52 MiB | 21.66 MiB/s, done.
Resolving deltas: 100% (278/278), done.


In [ ]:
!ls -al qvina/bin

total 22660
drwxr-xr-x 2 root root    4096 Nov 28 17:23 .
drwxr-xr-x 8 root root    4096 Nov 28 17:23 ..
-rwxr-xr-x 1 root root 3317424 Nov 28 17:23 qvina02
-rw-r--r-- 1 root root 4277120 Nov 28 17:23 qvina2.1
-rwxr-xr-x 1 root root 1979191 Nov 28 17:23 qvina_split02
-rw-r--r-- 1 root root 4279488 Nov 28 17:23 qvina-w
-rwxr-xr-x 1 root root 4033824 Nov 28 17:23 qvina-w_serial
-rwxr-xr-x 1 root root 3317424 Nov 28 17:23 vina
-rwxr-xr-x 1 root root 1979191 Nov 28 17:23 vina_split


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
import glob
import subprocess
from concurrent.futures import ThreadPoolExecutor
import time

# --- 1. 사용자 설정 (qvina (1).ipynb 기반) ---
BASE_DIR = '/content/drive/MyDrive/'

# 파일 경로
RECEPTOR_NAME = 'vina_1nny.receptor.pdbqt'
RECEPTOR_PATH = os.path.join(BASE_DIR, RECEPTOR_NAME)
LIGANDS_DIR_DRIVE = os.path.join(BASE_DIR, 'ligand/') # Drive 소스 폴더 (리간드 검색에만 사용)
RESULTS_DIR_DRIVE = os.path.join(BASE_DIR, 'results_qvina/') # Drive 결과 폴더

# Colab 로컬 디스크 임시 공간
TEMP_DIR = '/content/temp_qvina_local/'
# LIGANDS_DIR_LOCAL 정의는 유지하되, 리간드 복사 로직은 제거됨
LIGANDS_DIR_LOCAL = os.path.join(TEMP_DIR, 'ligands/')
RECEPTOR_PATH_LOCAL = os.path.join(TEMP_DIR, RECEPTOR_NAME)

# 도킹 파라미터 (config.txt 수치 적용)
CENTER_X, CENTER_Y, CENTER_Z = 31.3, 28.6, 21.5
SIZE_X, SIZE_Y, SIZE_Z = 24.0, 16.0, 15.0
EXHAUSTIVENESS = 8
QVINA_SEED = 1234
NUM_MODES = 1
CONFIG_PATH = '/content/config.txt' # 로컬에 설정 파일 생성

# --- 2. 폴더 생성 및 파일 복사 (리간드 복사 제외) ---
print("1. 폴더 생성 및 수용체 파일 복사 시작...")

# 로컬 임시 폴더 초기화 및 생성
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
# LIGANDS_DIR_LOCAL 폴더 생성은 유지 (나중에 검색 시 필요)
os.makedirs(LIGANDS_DIR_LOCAL, exist_ok=True)
os.makedirs(RESULTS_DIR_DRIVE, exist_ok=True)

# 수용체 파일 로컬 복사
try:
    shutil.copy(RECEPTOR_PATH, RECEPTOR_PATH_LOCAL)
    print(f"   -> 수용체 파일 복사 완료: {RECEPTOR_PATH_LOCAL}")
except FileNotFoundError:
    print(f"❌ 오류: 수용체 파일 '{RECEPTOR_PATH}'를 찾을 수 없습니다. 경로를 확인하세요.")
    raise

# ----------------------------------------------------
# 🚫 리간드 파일 복사 로직 (lig_files 관련 전체 블록) 제거됨
# ----------------------------------------------------

# --- 3. 설정 파일 (config.txt) 로컬 생성 ---
config_content = f"""
receptor = {RECEPTOR_PATH_LOCAL}
center_x = {CENTER_X}
center_y = {CENTER_Y}
center_z = {CENTER_Z}
size_x = {SIZE_X}
size_y = {SIZE_Y}
size_z = {SIZE_Z}
num_modes = {NUM_MODES}
exhaustiveness = {EXHAUSTIVENESS}
seed = {QVINA_SEED}
"""
with open(CONFIG_PATH, 'w') as f:
    f.write(config_content)
print(f"2. 설정 파일 생성 완료: {CONFIG_PATH}")

# 참고: Step 3 실행 시 리간드 파일을 LIGANDS_DIR_DRIVE에서 직접 검색하고 읽도록 수정해야 합니다.

1. 폴더 생성 및 수용체 파일 복사 시작...
   -> 수용체 파일 복사 완료: /content/temp_qvina_local/vina_1nny.receptor.pdbqt
2. 설정 파일 생성 완료: /content/config.txt


In [ ]:
!unzip ligands/all_ligand.zip -d temp_qvina_local/ligand

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: temp_qvina_local/ligand/lig_5500.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5501.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5502.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5503.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5504.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5505.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5506.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5507.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5508.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5509.pdbqt  
  inflating: temp_qvina_local/ligand/lig_550.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5510.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5511.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5512.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5513.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5514.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5515.pdbqt  
  inflating: temp_qvina_local/ligand/lig_5516.p

In [ ]:
!qvina02 --help

/bin/bash: line 1: qvina02: command not found


In [ ]:
# 1. 이전 qvina 폴더 제거 및 재설치
!rm -rf qvina
!git clone https://github.com/QVina/qvina.git
!chmod +x qvina/bin/qvina02

# 2. qvina02 실행 파일을 시스템이 찾는 경로로 복사
# /usr/local/bin은 시스템이 명령어를 찾는 일반적인 경로입니다.
!cp qvina/bin/qvina02 /usr/local/bin/

print("✅ qvina02 실행 파일 복사 완료. 이제 시스템이 명령어를 찾을 수 있습니다.")

Cloning into 'qvina'...
remote: Enumerating objects: 526, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 526 (delta 1), reused 15 (delta 0), pack-reused 510 (from 1)
Receiving objects: 100% (526/526), 27.52 MiB | 12.97 MiB/s, done.
Resolving deltas: 100% (278/278), done.
✅ qvina02 실행 파일 복사 완료. 이제 시스템이 명령어를 찾을 수 있습니다.


In [ ]:
import subprocess
from concurrent.futures import ThreadPoolExecutor
import time
import os
import glob
import shutil

# --- 4. 도킹 실행 함수 (로그 저장 기능 추가) ---
def run_qvina_docking(ligand_drive_path):
    """단일 리간드 도킹을 실행하고 결과 파일과 로그 파일을 Drive에 저장합니다."""

    name = os.path.basename(ligand_drive_path).replace('.pdbqt', '')

    # Drive 최종 출력 파일 및 로그 파일 경로 정의
    final_out = os.path.join(RESULTS_DIR_DRIVE, f'{name}_out.pdbqt')
    final_log = os.path.join(RESULTS_DIR_DRIVE, f'{name}.log') # 🌟 Drive에 로그 파일 저장

    if os.path.exists(final_out):
        return f"SKIP: {name}"

    # 로컬 임시 파일 경로
    temp_out = os.path.join(TEMP_DIR, f'{name}_out.pdbqt')
    temp_log = os.path.join(TEMP_DIR, f'{name}.log') # 🌟 임시 로그 파일 경로

    # QVina 2 명령어 구성
    command = [
        'qvina02',
        '--config', CONFIG_PATH,
        '--ligand', ligand_drive_path,
        '--out', temp_out,
        '--log', temp_log # 🌟 --log 옵션 추가
    ]

    try:
        # 명령어 실행 (파일로 로그를 출력하므로, subprocess는 캡처하지 않습니다.)
        result = subprocess.run(command, check=True, timeout=600, capture_output=True, text=True)

        # 성공 확인 및 파일 복사
        if os.path.exists(temp_out) and os.path.getsize(temp_out) > 1000:
            shutil.copy(temp_out, final_out)
            shutil.copy(temp_log, final_log) # 🌟 로컬 로그 파일을 Drive로 복사

            # 친화도 값 추출 (로그 파일 대신 캡처된 stdout에서 추출)
            affinity_line = result.stdout.splitlines()[-2]
            affinity = affinity_line.split()[1] if len(affinity_line.split()) > 1 else 'N/A'

            return f"SUCCESS: {name} | Affinity: {affinity} kcal/mol | Log Saved"
        else:
            return f"FAILED (Output Check): {name} | Output file is missing or too small."

    except subprocess.CalledProcessError as e:
        error_msg = e.stderr.strip().splitlines()[-1] if e.stderr else 'Unknown error'
        return f"ERROR (Subprocess): {name} | Failed: {error_msg}"
    except subprocess.TimeoutExpired:
        return f"ERROR (Timeout): {name} | Docking exceeded 120 seconds."
    except Exception as e:
        return f"ERROR (Other): {name} | {e}"

# --- 5. 메인 실행 블록 ---
if __name__ == '__main__':
    start_time = time.time()

    # 리간드 파일을 Drive 경로에서 직접 검색
    all_ligands = glob.glob(os.path.join(LIGANDS_DIR_DRIVE, 'lig_*.pdbqt'))

    if not all_ligands:
        print(f"\n❌ FATAL ERROR: Drive 경로 '{LIGANDS_DIR_DRIVE}'에서 'lig_*.pdbqt' 파일을 찾지 못했습니다. 경로를 확인하세요.")
    else:
        MAX_WORKERS = os.cpu_count() or 4
        print(f"\n[INFO] 총 {len(all_ligands)}개의 리간드를 {MAX_WORKERS}개 코어로 병렬 도킹을 시작합니다.")

        # ThreadPoolExecutor를 사용하여 병렬 실행
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            results = list(executor.map(run_qvina_docking, all_ligands))

        # --- 6. 결과 요약 ---

        elapsed_time = time.time() - start_time
        success_count = sum(1 for r in results if r.startswith("SUCCESS"))
        error_count = sum(1 for r in results if r.startswith("ERROR") or r.startswith("FAILED"))
        skip_count = sum(1 for r in results if r.startswith("SKIP"))

        print("\n" + "="*60)
        print("✨ 도킹 작업 완료 요약")
        print(f"총 리간드 수: {len(all_ligands)}")
        print(f"✅ 성공적으로 도킹 완료: {success_count} 개")
        print(f"⏭️  이전에 완료되어 건너뜀: {skip_count} 개")
        print(f"❌ 오류 발생: {error_count} 개")
        print(f"⏱️  총 소요 시간: {elapsed_time/3600:.2f} 시간 ({elapsed_time:.0f} 초)")
        print("="*60)

        error_count = sum(1 for r in results if r.startswith("ERROR") or r.startswith("FAILED"))
        skip_count = sum(1 for r in results if r.startswith("SKIP"))

        # ⭐ 여기에 추가: 오류 발생 리간드 이름 목록 추출
        error_ligands = [r for r in results if r.startswith("ERROR") or r.startswith("FAILED")]
        # ⭐ 이름과 간단한 오류 메시지 출력
        if error_ligands:
            print("\n--- 🚨 오류 발생 리간드 상세 목록 🚨 ---")
            # 최대 10개만 출력하거나 전체를 출력 (전체가 너무 길면 10개로 제한 권장)
            for i, error_info in enumerate(error_ligands):
                if i < 30: # 상위 10개만 출력 (로그가 길어지는 것 방지)
                    print(f"[FAIL {i+1}] {error_info}")
            if len(error_ligands) >30:
                print(f"... 외 {len(error_ligands) - 10}개의 리간드에서 오류 발생.")

        print("\n" + "="*60)


[INFO] 총 10005개의 리간드를 8개 코어로 병렬 도킹을 시작합니다.

✨ 도킹 작업 완료 요약
총 리간드 수: 10005
✅ 성공적으로 도킹 완료: 0 개
⏭️  이전에 완료되어 건너뜀: 9999 개
❌ 오류 발생: 6 개
⏱️  총 소요 시간: 0.00 시간 (1 초)

--- 🚨 오류 발생 리간드 상세 목록 🚨 ---
[FAIL 1] ERROR (Subprocess): lig_7519 | Failed: Thank you!
[FAIL 2] ERROR (Subprocess): lig_6323 | Failed: Thank you!
[FAIL 3] ERROR (Subprocess): lig_6325 | Failed: Thank you!
[FAIL 4] ERROR (Subprocess): lig_6326 | Failed: Thank you!
[FAIL 5] ERROR (Subprocess): lig_5055 | Failed: Thank you!
[FAIL 6] ERROR (Subprocess): lig_1582 | Failed: Thank you!



In [ ]:
import os
import shutil
from datetime import datetime
from google.colab import files
import time

# --- 1. 경로 설정 ---
# ⚠️ 커서가 위치한 Drive의 원본 폴더 경로
DRIVE_SOURCE_FOLDER = "/content/drive/MyDrive/results_qvina/"
# ZIP 파일을 임시로 생성할 로컬 디스크 경로
ZIP_NAME = "qvina_output_archive"
ZIP_PATH_LOCAL = f"/content/{ZIP_NAME}"


print(f"--- 1. 압축 작업 시작 (대상: {DRIVE_SOURCE_FOLDER}) ---")
start_time = time.time()

try:
    if not os.path.isdir(DRIVE_SOURCE_FOLDER):
        raise FileNotFoundError(f"경로를 찾을 수 없습니다: {DRIVE_SOURCE_FOLDER}")

    # 2. Drive 폴더를 ZIP 파일로 압축 (로컬 디스크에 임시 생성)
    # root_dir: 압축할 상위 디렉토리 (MyDrive/results_qvina)
    # base_dir: 압축할 대상 폴더 이름 (qvina)
    shutil.make_archive(
        base_name=ZIP_PATH_LOCAL,
        format='zip',
        root_dir=os.path.dirname(DRIVE_SOURCE_FOLDER),
        base_dir=os.path.basename(DRIVE_SOURCE_FOLDER)
    )

    final_zip_file = f"{ZIP_PATH_LOCAL}.zip"

    print(f"✅ 압축 완료 (소요 시간: {time.time() - start_time:.2f}초)")
    print(f"생성된 ZIP 파일 크기: {os.path.getsize(final_zip_file) / (1024*1024):.2f} MB")


    # --- 3. 로컬 컴퓨터로 다운로드 ---
    print("\n--- 3. 로컬 컴퓨터로 다운로드 시작 ---")

    # Colab files 모듈을 사용하여 브라우저 다운로드를 요청합니다.
    files.download(final_zip_file)

    print("✅ 다운로드 요청 완료. 브라우저에서 다운로드 상태를 확인하세요.")

except FileNotFoundError as e:
    print(f"❌ FATAL ERROR: 지정된 Drive 경로를 찾을 수 없습니다. Drive 연결을 확인해주세요.")
except Exception as e:
    print(f"❌ 오류 발생: {e}")

--- 1. 압축 작업 시작 (대상: /content/drive/MyDrive/results_qvina/) ---


KeyboardInterrupt: 

In [ ]:
import os
import glob
import pandas as pd

# --- 경로 설정 (이전 코드와 동일) ---
BASE_DIR = "/content/drive/MyDrive/"
RESULTS_DIR_DRIVE = f"{BASE_DIR}/results_qvina"

# ❌ 오류로 제외할 리간드 이름 목록
EXCLUDE_LIGANDS = [
    'lig_7519', 'lig_6323', 'lig_6325', 'lig_6326', 'lig_5055', 'lig_1582'
]

OUTPUT_CSV_FILE = os.path.join(RESULTS_DIR_DRIVE, "final_affinity_scores.csv")

print(f"✅ 결과 폴더: {RESULTS_DIR_DRIVE}")
print(f"✅ 제외 리간드 수: {len(EXCLUDE_LIGANDS)}개")

✅ 결과 폴더: /content/drive/MyDrive//results_qvina
✅ 제외 리간드 수: 6개


In [ ]:
import os
import glob
import pandas as pd
from typing import Tuple, Optional

# ======================================================
# 1. 경로 및 설정 정의 (이 부분은 노트북 상단 설정과 일치해야 합니다)
# ======================================================

# ⚠️ 이 경로는 Drive에 있는 'docking' 폴더가 MyDrive 바로 아래에 있을 경우의 예시입니다.
BASE_DIR = "/content/drive/MyDrive/"

# QuickVina 결과 파일이 저장된 폴더 경로
RESULTS_DIR_DRIVE = f"{BASE_DIR}/results_qvina"

# ❌ 오류로 제외할 리간드 이름 목록 (파일 이름 기준)
EXCLUDE_LIGANDS = [
    'lig_7519', 'lig_6323', 'lig_6325', 'lig_6326', 'lig_5055', 'lig_1582'
]

OUTPUT_CSV_FILE = os.path.join(RESULTS_DIR_DRIVE, "final_affinity_scores_with_DBID.csv")

print(f"✅ 결과 폴더: {RESULTS_DIR_DRIVE}")
print(f"✅ 제외 리간드 수: {len(EXCLUDE_LIGANDS)}개")


# ======================================================
# 2. Affinity Score 및 DB ID 추출 함수
# ======================================================
def extract_affinity_score_and_id(file_path: str) -> Tuple[Optional[str], Optional[float]]:
    """
    *_out.pdbqt 파일에서 REMARK Name (DB ID)과 Affinity Score를 추출합니다.
    """

    # 파일명 추출 (제외 목록 체크용)
    file_base_name = os.path.basename(file_path).replace('_out.pdbqt', '')

    # ❌ 오류 목록에 있는 파일은 건너뜁니다.
    if file_base_name in EXCLUDE_LIGANDS:
        return None, None

    affinity_score = None
    ligand_id = None

    try:
        with open(file_path, 'r') as f:
            for line in f:
                # 1. Affinity Score 추출 (REMARK VINA RESULT 라인)
                if 'REMARK VINA RESULT' in line:
                    # 두 번째 줄의 네 번째 항목이 친화도 점수입니다.
                    try:
                        affinity_score = float(line.split()[3])
                    except (IndexError, ValueError):
                        affinity_score = 'N/A (Score Parse Error)'

                # 2. 리간드 DB ID 추출 (REMARK Name 라인)
                if 'REMARK  Name =' in line:
                    # 'REMARK  Name =' 뒤의 공백과 ID를 추출
                    ligand_id = line.split('=')[1].strip()

                # 두 정보를 모두 찾았으면 루프 종료 (효율성)
                if affinity_score is not None and ligand_id is not None:
                    break

        if affinity_score is None or affinity_score == 'N/A (Score Parse Error)':
            # 점수를 찾지 못했으나 ID는 찾은 경우
            return ligand_id, 'N/A (Score Missing)'

        # ID를 찾지 못했으나 점수는 찾은 경우 (파일 이름으로 대체)
        if ligand_id is None:
             ligand_id = file_base_name

        return ligand_id, affinity_score

    except Exception:
        # 파일 읽기 오류 등
        return file_base_name, 'N/A (File Read Error)'


# ======================================================
# 3. 메인 추출 루프 및 CSV 저장
# ======================================================

print("\n--- 3. Affinity Score 추출 및 CSV 저장 시작 ---")

# Drive 결과 폴더 내 모든 _out.pdbqt 파일 검색
all_out_files = glob.glob(os.path.join(RESULTS_DIR_DRIVE, 'results_qvina/*_out.pdbqt'))
data = []
success_count = 0

if not all_out_files:
    # results_qvina/results_qvina/ 경로에 저장되었을 가능성을 대비하여 경로 재검사
    all_out_files = glob.glob(os.path.join(RESULTS_DIR_DRIVE, '*_out.pdbqt'))

for file_path in all_out_files:
    ligand_id, affinity = extract_affinity_score_and_id(file_path)

    if ligand_id is not None:
        data.append({'Ligand_DB_ID': ligand_id, 'Affinity_kcal/mol': affinity})
        success_count += 1

if not data:
    print("❌ 추출된 데이터가 없습니다. *_out.pdbqt 파일 경로를 확인하거나, 파일이 비어있는지 확인하세요.")
else:
    # DataFrame으로 변환 및 Affinity Score 기준으로 정렬
    df_results = pd.DataFrame(data)

    # Affinity Score가 숫자(float)인 항목만 정렬 (낮을수록 좋음)
    df_results['Affinity_kcal/mol_NUM'] = pd.to_numeric(df_results['Affinity_kcal/mol'], errors='coerce')
    df_results_sorted = df_results.sort_values(by='Affinity_kcal/mol_NUM', ascending=True)

    # 최종 출력 CSV 파일에는 NUM 컬럼 제외
    df_final_output = df_results_sorted.drop(columns=['Affinity_kcal/mol_NUM'])

    # CSV 파일로 저장
    df_final_output.to_csv(OUTPUT_CSV_FILE, index=False)

    print("\n--- ✅ 작업 완료 ---")
    print(f"총 성공적으로 추출 및 저장된 리간드 수: {len(df_final_output)}개")
    print(f"⭐ 최종 CSV 파일 저장 위치: {OUTPUT_CSV_FILE}")
    print(f"⭐ 가장 낮은 Affinity Score (최상위 후보): {df_final_output.iloc[0]['Affinity_kcal/mol']} kcal/mol")

✅ 결과 폴더: /content/drive/MyDrive//results_qvina
✅ 제외 리간드 수: 6개

--- 3. Affinity Score 추출 및 CSV 저장 시작 ---

--- ✅ 작업 완료 ---
총 성공적으로 추출 및 저장된 리간드 수: 9999개
⭐ 최종 CSV 파일 저장 위치: /content/drive/MyDrive//results_qvina/final_affinity_scores_with_DBID.csv
⭐ 가장 낮은 Affinity Score (최상위 후보): -9.9 kcal/mol


In [ ]:
import pandas as pd
import os

# --- 1. 파일 경로 설정 ---
# 🚨 파일 이름이 변경되었을 수 있으니, 경로가 정확한지 확인하세요.
AFFINITY_FILE = "final_affinity_scores_with_DBID.csv"
SMILES_FILE = "final_merged_10000_candidates.smi"
OUTPUT_FILE = "final_candidates_with_score_smiles.csv"

# --- 2. 데이터 로드 ---

# 2.1. Affinity Score 데이터 로드 (CSV)
df_affinity = pd.read_csv(AFFINITY_FILE)
# Ligand_DB_ID 컬럼을 병합 키로 사용합니다.
df_affinity.rename(columns={'Ligand_DB_ID': 'ID'}, inplace=True)

# 2.2. SMILES 데이터 로드 (SMI 파일 - 탭 구분)
# 이 파일은 smi, ID, (Score_FP_main), (Score_2), (Score_3), (Score_4), pharma_sig 순서로 되어 있습니다.
# 컬럼 이름이 유동적일 수 있으므로, ID와 SMILES만 명확히 지정하여 로드합니다.
try:
    # Read SMILES, ID, and ensure only necessary columns are kept
    df_smiles = pd.read_csv(SMILES_FILE, sep='\t', header=None, names=['smi', 'ID'] + list(range(10)))
    df_smiles = df_smiles[['ID', 'smi']]

except Exception as e:
    # 탭 구분이 아닌 경우 대비 (오류 발생 시 CSV로 재시도 가능)
    print(f"❌ SMILES 파일 로드 중 오류 발생: {e}. 데이터 구조를 확인하세요.")
    sys.exit()

# --- 3. ID를 기준으로 데이터 병합 ---
# ID 컬럼을 기준으로 Inner Join (두 데이터 모두에 있는 리간드만 남김)
df_merged = pd.merge(
    df_affinity,
    df_smiles,
    on='ID',
    how='inner'
)

# 4. 불필요한 컬럼 정리 및 정렬
# (Affinity Score 기준으로 정렬: 낮을수록 좋음)
df_merged.sort_values(by='Affinity_kcal/mol', ascending=True, inplace=True)

# 5. 최종 CSV 파일로 저장
df_merged.to_csv(OUTPUT_FILE, index=False)

print("\n--- ✅ 통합 완료 ---")
print(f"총 통합된 최종 후보군 수: {len(df_merged)}개")
print(f"⭐ 최종 파일 저장 위치: {OUTPUT_FILE}")
print("이제 이 파일의 SMILES 컬럼을 사용하여 독성 예측 모델을 구동할 수 있습니다.")


--- ✅ 통합 완료 ---
총 통합된 최종 후보군 수: 9999개
⭐ 최종 파일 저장 위치: final_candidates_with_score_smiles.csv
이제 이 파일의 SMILES 컬럼을 사용하여 독성 예측 모델을 구동할 수 있습니다.


## 독성 예측

In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 77.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import kagglehub
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import time

# --- 파일 및 설정 정의 ---
CANDIDATES_FILE = "final_candidates_with_score_smiles.csv"
OUTPUT_FILE_FINAL = "final_candidates_with_toxicity.csv"
target = 'NR-AhR'

# Morgan fingerprint 설정 (이전에 학습에 사용된 것과 동일)
morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=1024)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 1. 모델 정의 ---
class MLP(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)

# --- 2. 모델 재학습 (Tox21 데이터) ---
print("--- 1. 독성 예측 모델 재학습 시작 ---")
path = kagglehub.dataset_download("epicskills/tox21-dataset")
df_tox21 = pd.read_csv(path + "/tox21.csv")

# 학습 데이터 전처리 (이전과 동일)
df_ar = df_tox21[['smiles', target]].dropna(subset=[target])

def mol_to_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return morgan_gen.GetFingerprintAsNumPy(mol)

fps = []
valid_idx = []
for i, smi in enumerate(df_ar['smiles']):
    fp = mol_to_fp(smi)
    if fp is not None:
        fps.append(fp)
        valid_idx.append(i)

X_train_full = np.array(fps)
y_train_full = df_ar[target].iloc[valid_idx].values.astype(int)

# 학습을 위한 데이터셋 정의
class Tox21TrainDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_data = Tox21TrainDataset(X_train_full, y_train_full)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# 모델 학습 설정 (이전과 동일)
model = MLP().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
EPOCHS = 10

# 학습 루프
for epoch in range(EPOCHS):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device).unsqueeze(1)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()

print("✅ 모델 학습 완료.")

--- 1. 독성 예측 모델 재학습 시작 ---


100%|██████████| 126k/126k [00:00<00:00, 442kB/s]

Extracting files...



[02:14:11] WARNING: not removing hydrogen atom without neighbors
[02:14:11] Explicit valence for atom # 3 Al, 6, is greater than permitted
[02:14:11] Explicit valence for atom # 4 Al, 6, is greater than permitted
[02:14:11] Explicit valence for atom # 4 Al, 6, is greater than permitted
[02:14:11] Explicit valence for atom # 9 Al, 6, is greater than permitted
[02:14:11] Explicit valence for atom # 5 Al, 6, is greater than permitted
[02:14:12] Explicit valence for atom # 16 Al, 6, is greater than permitted
[02:14:12] Explicit valence for atom # 20 Al, 6, is greater than permitted


✅ 모델 학습 완료.


In [ ]:
import pandas as pd
import os
import sys

# --- 파일 경로 설정 ---
INPUT_FILE = "final_candidates_with_toxicity.csv"
OUTPUT_FILE_TOP20 = "top4700_filtered.csv"

# --- 설정 임계값 ---
# NR-AhR 독성 확률 20% 초과시 탈락 (안전성 필터)
TOXICITY_THRESHOLD = 0.2
# 최종 선택할 후보의 개수
TOP_N = 4700

# --- 데이터 로드 ---
try:
    df_final = pd.read_csv(INPUT_FILE)
except FileNotFoundError:
    print(f"❌ 오류: 파일 '{INPUT_FILE}'을 찾을 수 없습니다. 독성 예측 코드를 먼저 실행하여 파일을 생성해야 합니다.")
    sys.exit()

# --- 1. 데이터 정리 및 형 변환 ---
# Affinity score와 독성 확률을 숫자로 변환 (N/A 값은 무시)
df_final['Affinity_kcal/mol_NUM'] = pd.to_numeric(df_final['Affinity_kcal/mol'], errors='coerce')
df_final['NR_AhR_Toxicity_Prob'] = pd.to_numeric(df_final['NR_AhR_Toxicity_Prob'], errors='coerce')

# 결측값(NaN)이 있는 행은 제외하고 유효한 데이터만 사용합니다.
df_valid = df_final.dropna(subset=['Affinity_kcal/mol_NUM', 'NR_AhR_Toxicity_Prob'])


# --- 2. 1단계 필터링: 독성 임계값으로 컷 (Prob <= 0.2) ---
print(f"🔥 1단계 필터링: 독성 확률 {TOXICITY_THRESHOLD} 이하인 후보 선별...")

# 독성 확률이 임계값 이하인 후보만 선택 (안전성 확보)
df_safe = df_valid[df_valid['NR_AhR_Toxicity_Prob'] <= TOXICITY_THRESHOLD].copy()

if len(df_safe) == 0:
    print("❌ 경고: 독성 임계값(0.2)을 통과한 안전한 후보가 없습니다.")
    sys.exit()

print(f"✅ 필터링 후 안전 후보군 수: {len(df_safe)}개")


# --- 3. 2단계 정렬 및 Top N 선택 ---
# 1순위: 결합 친화도 (Affinity_kcal/mol_NUM) - 오름차순 (낮을수록 좋음)
# 2순위: 독성 확률 (NR_AhR_Toxicity_Prob) - 오름차순 (낮을수록 좋음)
df_best_candidates = df_safe.sort_values(
    by=['Affinity_kcal/mol_NUM', 'NR_AhR_Toxicity_Prob'],
    ascending=[True, True]
)

# Top 20 선택
df_top20 = df_best_candidates.head(TOP_N)

# --- 4. 최종 CSV 저장 ---
# 불필요한 NUM 컬럼 제거 후 저장
df_output = df_top20.drop(columns=['Affinity_kcal/mol_NUM'])

df_output.to_csv(OUTPUT_FILE_TOP20, index=False)


print("\n--- ✅ 최종 Top 20 후보군 선별 완료 ---")
print(f"⭐ 최종 파일 저장 위치: {OUTPUT_FILE_TOP20}")
print("\n⭐ 선택된 Top 20 후보 (Affinity가 낮고 독성이 낮은 순서):")
print(df_output)

🔥 1단계 필터링: 독성 확률 0.2 이하인 후보 선별...
✅ 필터링 후 안전 후보군 수: 4770개

--- ✅ 최종 Top 20 후보군 선별 완료 ---
⭐ 최종 파일 저장 위치: top4700_filtered.csv

⭐ 선택된 Top 20 후보 (Affinity가 낮고 독성이 낮은 순서):
              ID  Affinity_kcal/mol  \
19     Z20120134               -9.2   
20    Z224889030               -9.2   
26    Z243556184               -9.1   
27     Z15942268               -9.1   
28     Z13601298               -9.1   
...          ...                ...   
9892   Z19025999               -6.0   
9893  Z756843474               -6.0   
9894   Z15860853               -6.0   
9895   Z16377259               -6.0   
9896  Z195916940               -6.0   

                                                    smi  NR_AhR_Toxicity_Prob  
19    O=C(CSc1nnc(-c2ccccc2F)n1-c1ccccc1)OCn1nnc2ccc...          3.220311e-06  
20    O=C(CCc1nc(-c2ccccc2)c(-c2ccccc2)o1)NCc1nnc2cc...          5.696140e-02  
26    O=C(CCn1c(=O)[nH]c(=O)c2ccccc21)NCC1CC2c3ccccc...          2.693100e-07  
27    Cc1cc(=O)oc2cc(OCc3ccc(C(=O)OCC(=O)NC